## Classiq Implementation For The Research Paper: Symmetry enhanced variational quantum spin eigensolver

In [109]:
# Handle imports
from classiq import * # Using Classiq Version 0.52.0 Release
import numpy as np
from typing import cast, List
import quimb as qu
from functools import reduce
from operator import mul
from classiq.execution import ExecutionSession
# import scipy as sp

In [110]:
# Circuit design variables
n_bits = 4
n_layers = 5
J = 1

# Dummy parameters for testing implementation of ciruit
theta_1 = np.pi
theta_2 = np.pi
theta_3 = np.pi
theta_4 = np.pi
theta_5 = np.pi
theta_6 = np.pi
theta_7 = np.pi

training_params = [theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7]

# Initialization circuit layer
@qfunc
def Init(q: QArray) -> None:
    # First, do PauliX on all qubits
    for i in range(n_bits):
        X(q[i])

    # Next, add the Hadamards and CNOT
    for i in range(0, n_bits, 2):
        hadamard_transform(q[i])
        CX(q[i],q[i + 1])


# N block function
@qfunc
def N_block(q0: QBit, q1: QBit, layer: CReal) -> None:
    RZ(np.pi / 2, q1)

    CX(q1, q0)
     
    RZ(2 * layer - np.pi/2, q0)
    RY(np.pi / 2 - 2 * layer, q1)
    
    CX(q0, q1)

    RY(2 * layer - np.pi/2, q1)

    CX(q1, q0)
    RZ(-np.pi / 2, q0)


# Main ansatz layer
@qfunc
def Sz_conserving_layer(q: QArray, params: CArray[CReal]) -> None:
    # Apply the N_block layer for entangling
    N_block(q[0],q[1], params[0])
    N_block(q[2],q[3], params[1])
    N_block(q[1],q[2], params[2])

    # # Apply the Phase Gates
    for i in range(n_bits):
        PHASE(params[3+i], q[i])

# Function to build the Paulis string
def build_pauli_string(N: int, J: float) -> list[tuple[str, float]]:
    pauli_list = []
    
    # Loop over each neighboring pair of spins (i, i+1)
    for i in range(N-1):
        # Create Pauli strings for sigma_x, sigma_y, sigma_z interactions
        for pauli in ['X', 'Y', 'Z']:
            # Identity for other positions, Pauli for position i and i+1
            identity = ['I'] * N
            identity[i] = pauli
            identity[i+1] = pauli
            
            # Join Pauli string and add it to the list with coefficient J
            pauli_string = ''.join(identity)
            pauli_list.append((pauli_string, J))
    
    return pauli_list

# Pauli matrices as numpy arrays
PAULI_MATRICES = {
    "I": np.array([[1, 0], [0, 1]]),  # Identity
    "X": np.array([[0, 1], [1, 0]]),  # Pauli-X
    "Y": np.array([[0, -1j], [1j, 0]]),  # Pauli-Y
    "Z": np.array([[1, 0], [0, -1]])   # Pauli-Z
}

def pauli_str_to_matrix(pauli_str):
    """
    Convert a Pauli string to the corresponding matrix using tensor product.
    Example: 'XXI' -> X ⊗ X ⊗ I
    """
    matrices = [PAULI_MATRICES[p] for p in pauli_str]  # List of Pauli matrices
    return reduce(np.kron, matrices)  # Apply tensor product (Kronecker product)

def hamiltonian_to_matrix(pauli_list):
    """
    Convert a list of Pauli terms (with coefficients) to a Hamiltonian matrix.
    Args:
        pauli_list (list of tuples): Each tuple contains a Pauli string and its coefficient.
    Returns:
        np.ndarray: The full Hamiltonian matrix.
    """
    N = 2 ** len(pauli_list[0][0])  # Dimension of the matrix (2^N for N qubits)
    H = np.zeros((N, N), dtype=complex)  # Initialize the Hamiltonian matrix
    
    for pauli_str, coeff in pauli_list:
        # Convert Pauli string to matrix and multiply by its coefficient
        H += coeff * pauli_str_to_matrix(pauli_str)
    
    return H

# Definition of Pauli Terms
CHAR_TO_STUCT_DICT = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

# Functions for creating the Hamiltonian from the Paulis String
def pauli_str_to_enums(pauli):
    return [CHAR_TO_STUCT_DICT[s] for s in pauli]

def pauli_list_to_hamiltonian(pauli_list):
    return [
        PauliTerm(
            pauli=pauli_str_to_enums(pauli), coefficient=cast(complex, coeff).real
        )
        for pauli, coeff in pauli_list
    ]


 # Create the pauli list
pauli_list = build_pauli_string(n_bits,J)
# Create the Hamiltonian
heis_ham = pauli_list_to_hamiltonian(pauli_list)
hamiltonian_matrix = hamiltonian_to_matrix(pauli_list)
# print(hamiltonian_matrix)
# Test
ham_matrix = qu.ham_heis(n_bits, J, cyclic=False)
# print(ham_matrix)
# Main Program
@qfunc
def main(q: Output[QArray], p: CArray[CReal,7]) -> None:
    # Create the array of bits of proper size
    # q = QArray("q")
    allocate(n_bits, q)
    
    # Do the initialization circuit
    Init(q)

    # Do n layers of the Sz_conserving Ansatz
    Sz_conserving_layer(q,p)
    # for _ in range(n_layers):
    #     Sz_conserving_layer(q,params)


In [111]:
# Simulation
mod = create_model(main)
qprog = synthesize(mod)
# show(qprog)

execution_session = ExecutionSession(qprog)

estimate_result = execution_session.estimate(heis_ham, {"p": training_params})
print(estimate_result)

value=(2.0029296875+0j) variance=(-1+0j) metadata=EstimationMetadata(shots=2048, remapped_qubits=False, input_qubit_map=None)
